In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix


In [16]:
def calc_cost_metric(y_pred, y_true, n_classes=5):
    """_summary_

    Args:
        y_pred (_type_): n numpy (1 dim, predictions after argmax)
        y_test (_type_): n numpy (1 dim)
    """

    r = np.arange(0,n_classes)
    n = y_pred.shape[0]
    cost_matrix = 2 * np.abs(r-r[:, None])+1
    print(cost_matrix)
    cm = confusion_matrix(y_pred=y_pred, y_true=y_true, labels = np.arange(0,n_classes))
    return (cm * cost_matrix).sum()/n


y_true = np.array([0,2,2])
y_pred = np.array([0,0,2])

calc_cost_metric(y_true, y_pred) 

[[1 3 5 7 9]
 [3 1 3 5 7]
 [5 3 1 3 5]
 [7 5 3 1 3]
 [9 7 5 3 1]]


2.3333333333333335

In [54]:
import os

len([os.stat(element).st_size for element in glob("./models/*.pt")])


26

In [32]:
import pandas as pd
buga = pd.read_csv("./mlruns/0/35efd3d08cf447ec91b920fa1e6e8bc6/artifacts/test_predictions_phase_8.csv", index_col=0)
buga.idxmax(axis=1).value_counts(normalize=True).reindex(["0","1","2","3","4"]).fillna(0)

0    0.000
1    0.136
2    0.862
3    0.002
4    0.000
dtype: float64

In [ ]:
import torch

In [18]:
import json

CONSTRAINTS_CONFIG = "./constraints_configs/config_015.json"

with open(CONSTRAINTS_CONFIG) as f:
    s= json.load(f)

torch.tensor(s, device= 1)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [9]:
import pandas as pd
import torch

import json



constraints = pd.read_csv("constraints_configs/config_003.csv", header=None)[0].values



array([1.  , 1.  , 1.  , 0.03, 1.  ])

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
#from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import (classification_model_densenet, 
                                                classification_model_resnet, 
                                                classification_model_vgg, 
                                                DummyModel
                                                )


from ordinaloss.utils.data_utils import data_load

from ordinaloss.utils.loss_utils import SinimLoss, GirlsLoss, CSCELoss
from ordinaloss.utils.loss_utils import create_ordinal_cost_matrix
from ordinaloss.utils.loss_utils import PredictionLoss, CombinedLoss

from ordinaloss.nextgen_engine import model_engine
from ordinaloss.utils.callback_utils import PrintingCallback

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

loaders, _, _ = data_load("../datasets/kneeKL224/", 16)


/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models
loaded ordinaloss.utils.data_utils
loaded ordinaloss.utils.loss_utils
loaded ordinaloss.utils.metric_utils!!
loaded ordinaloss.nextgen_engine.model_engine


In [4]:
constraints = torch.tensor([0.2, 1, 1, 1, 1])
test_dist   = torch.tensor([0.8, 0, 0, 0, 0.1])
current_lambdas = torch.ones(5)*0.2

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])

In [ ]:
model = DummyModel()

cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)
csce_loss = CSCELoss(cost_matrix)

init_lambdas = torch.tensor([0.2, 0.2, 0.2, 0.2, 0.2], device = device)
prediction_loss = PredictionLoss(lambdas = init_lambdas)

loss_fn = CombinedLoss(csce_loss, prediction_loss)

engine = model_engine.OrdinalEngine(
    model=model, loss_fn=loss_fn, 
    device=device, loaders=loaders, n_classes=5,
    use_lr_scheduler=False, callbacks=[],
    optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2)


In [ ]:
#model = classification_model_vgg("vgg16", num_classes = 5)

cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)



#callbacks = [PrintingCallback()]
callbacks = []

engine = model_engine.OrdinalEngine(
    model=model, loss_fn=CSCELoss(cost_matrix), 
    device=device, loaders=loaders, n_classes=5,
    use_lr_scheduler=False, callbacks=callbacks,
    optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2)

loaded ordinaloss.nextgen_engine.model_engine
